# Summarization using HuggingFace API

This notebook is used to generate summaries of the transcripts.

## Dependencies

### Virtual Environment Dependencies


The model evaluator leverages the huggingface library. The following packages must be installed:

1. transformers
2. sentencepiece
3. datasets
4. evaluate
5. torchmetrics


Due to the PEGASUS model, sentencepiece and a non cached transformers library must be installed.

In [ ]:
!pip install --no-cache-dir transformers sentencepiece
!pip install datasets evaluate torchmetrics

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 146.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 318.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 325.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 261.2 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 22.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.2/519.2 kB 49.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 14.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 1.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 66.6 MB/s eta 0:00:

### Imports

In [ ]:
import os
import json
from typing import Dict, List, Tuple, Callable
import pandas
from google.colab import drive, runtime
import sqlite3 as sq
from dataclasses import dataclass
import time
import transformers
from transformers import pipeline
from transformers.pipelines.pt_utils import KeyDataset
import torch
import timeit

In [ ]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [ ]:
drive.mount('/content/drive')

Mounted at /content/drive


## Paths

This notebook requires a connection with the database containing all of the transcript data. This can be generated by running the FinancialModelPrep [notebook](https://colab.research.google.com/drive/1IPQLXCfiAfVXH-W30vYevktjQZnV6qt6?usp=sharing) notebook to create the database and pull the financial data.

In [ ]:
project_path = "/content/drive/MyDrive/CSCI-5541/Project/"
data_path = os.path.join(project_path, "data")
model_path = os.path.join(project_path,"model")
summarization_model_path = os.path.join(model_path,"summarization")

In [ ]:
database_path = os.path.join(data_path, "earnings_transcripts_data.db")

In [ ]:
if not os.path.exists(summarization_model_path):
    os.makedirs(summarization_model_path)

## Data Classes and DB Util

### Data Classes

Objects to hold data from tables

In [ ]:
@dataclass
class Company:
    symbol: str

    def to_dict(self):
        return {
            "symbol": self.symbol
        }

@dataclass
class Transcript:
    symbol: str
    date: str
    year: int
    quarter: int
    transcript: str

    def to_dict(self):
        return {
            "symbol": self.symbol,
            "date": self.date,
            "year": self.year,
            "quarter": self.quarter,
            "transcript": self.transcript
        }


@dataclass
class Price:
    symbol: str
    date: str
    opening_price: float
    closing_price: float

    def to_dict(self):
        return {
            "symbol": self.symbol,
            "date": self.date,
            "opening_price": self.opening_price,
            "closing_price": self.closing_price,
        }

@dataclass
class Label:
    id: int
    symbol: str
    transcript_date: str
    price_day_of_meeting: float
    avg_value: float
    delta_days: int
    total_days: int
    avg_type: str
    label: str


@dataclass
class Predicted_Label:
    id: int
    true_label_id: int
    symbol: str
    transcript_date: str
    label: str

@dataclass
class TranscriptSummary:
    symbol: str
    date: str
    year: int
    quarter: int
    summarized_transcript: str
    summarization_method: str

    def to_dict(self):
        return {
            "symbol": self.symbol,
            "date": self.date,
            "year": self.year,
            "quarter": self.quarter,
            "summarized_transcript": self.summarized_transcript,
            "summarization_method": self.summarization_method,
        }

### DB Util
Utility for interacting with the database

In [ ]:
class DB_Util:
    def __init__(self, database_path: str):
        self.con = sq.connect(database_path)

    
    def create_or_drop_table(self, sql: str) -> bool:
        cursor = self.con.cursor()
        try:
            cursor.execute(
                sql,
            )
            return True
        except Exception as e:
            print(e)
            return False
        finally:
            self.con.commit()
            cursor.close()
            del cursor

    def execute_query(self, sql: str, args: List) -> List:
        cursor = self.con.cursor()
        try:
            query_results = cursor.execute(
                sql,
                args
            )
            return query_results.fetchall()
        except Exception as e:
            print(e)
            return []
        finally:
            cursor.close()
            del cursor


    def insert_data(self, sql: str, data: List) -> bool:
        cursor = self.con.cursor()
        try:
            cursor.execute(
                sql,
                data                
            )
            return True
        except Exception as e:
            print(e)
            return False
        finally:
            self.con.commit()
            cursor.close()
            del cursor
    
    def close_connection(self):
        self.con.close()
        del self.con

## Database Connection and Operations

### Create Database Connection
In order to interact with the database, simply pass the path to the database to the constructor.

In [ ]:
db_util = DB_Util(database_path=database_path)

### Creating Tables
Creating transcript summary table.

In [ ]:
table_definitions = [
    '''
        CREATE TABLE IF NOT EXISTS company(
            symbol CHAR[10] NOT NULL PRIMARY KEY
        )
    ''',
    '''           
    CREATE TABLE IF NOT EXISTS price(
        symbol CHAR[10] NOT NULL,
        date CHAR[10],
        opening_value FLOAT,
        closing_value FLOAT,
        PRIMARY KEY (symbol, date)
        FOREIGN KEY (symbol) REFERENCES company(symbol)
    )
    ''',
    '''
    CREATE TABLE IF NOT EXISTS transcript(
        symbol CHAR[10] NOT NULL,
        year INTEGER,
        quarter INTEGER,
        date CHAR[10],
        transcript CLOB,
        FOREIGN KEY (symbol) REFERENCES company(symbol),
        PRIMARY KEY(symbol, year, quarter, date)
    )
    ''',
    # Not imposing any constraints because we will probably need to experiment with this
    '''
    CREATE TABLE IF NOT EXISTS label(
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        symbol CHAR[10],
        transcript_date CHAR[10],
        price_day_of_meeting FLOAT,
        avg_value FLOAT,
        delta_days INTEGER,
        total_days INTEGER,
        avg_type CHAR[10],
        label CHAR[20],
        FOREIGN KEY (symbol) REFERENCES company
    )
    ''',
    '''
    CREATE TABLE IF NOT EXISTS transcript_summary(
        symbol CHAR[10] NOT NULL,
        year INTEGER,
        quarter INTEGER,
        date CHAR[10],
        summarized_transcript CLOB,
        summarization_method CHAR[80],
        FOREIGN KEY (symbol) REFERENCES company(symbol),
        PRIMARY KEY(symbol, year, quarter, date, summarization_method)
    )
    '''    
]

table_creation_status = [
    db_util.create_or_drop_table(
        sql=table_definition
    )
    for table_definition in table_definitions
]
print(table_creation_status)

[True, True, True, True, True]


### Queries

Queries for fetching transcripts and transcript summaries.

In [ ]:
def fetch_all_transcripts(db_util: str) -> List[Transcript]:
    transcript_records = db_util.execute_query(
        "SELECT * FROM transcript WHERE ?",
        [True]
    )
    return [
        Transcript(
            symbol=transcript_record[0],
            year=transcript_record[1],
            quarter=transcript_record[2],
            date=transcript_record[3],
            transcript=transcript_record[4],
        )
        for transcript_record in transcript_records
    ]

def fetch_all_summaries(db_util: str) -> List[Transcript]:
    transcript_records = db_util.execute_query(
        "SELECT * FROM transcript_summary WHERE ?",
        [True]
    )
    return [
        TranscriptSummary(
            symbol=transcript_record[0],
            year=transcript_record[1],
            quarter=transcript_record[2],
            date=transcript_record[3],
            summarized_transcript=transcript_record[4],
            summarization_method=transcript_record[5],
        )
        for transcript_record in transcript_records
    ]


def fetch_summaries_by_method(db_util: str, method: str) -> List[Transcript]:
    transcript_records = db_util.execute_query(
        "SELECT * FROM transcript_summary WHERE summarization_method = ?",
        [method]
    )
    return [
        TranscriptSummary(
            symbol=transcript_record[0],
            year=transcript_record[1],
            quarter=transcript_record[2],
            date=transcript_record[3],
            summarized_transcript=transcript_record[4],
            summarization_method=transcript_record[5],
        )
        for transcript_record in transcript_records
    ]


### Insert Summaries
Methods for inserting summaries into the database

In [ ]:
def add_summary(summary: TranscriptSummary, db_util: DB_Util)->bool:
        return db_util.insert_data(
            sql=''' INSERT INTO transcript_summary (symbol, year, quarter, date, summarized_transcript, summarization_method) Values(?, ?, ?, ?, ?, ?) ''',
            data= [summary.symbol, summary.year, summary.quarter, summary.date, summary.summarized_transcript, summary.summarization_method]
        )



def add_summaries(summaries: List[TranscriptSummary], db_util: DB_Util):
    for summary in summaries:
        status = add_summary(summary, db_util)
        if not status:
            print(f"Unable to add transcript {summary.symbol} {summary.date}")

## Generate Summarization Pipeline From HuggingFace

Run the cells below to create the summarization pipeline. Note that this loads the model to a gpu device if there is one available.

In [ ]:
# summarizer_name = "philschmid/tf-distilbart-cnn-12-6"
summarizer_name = "human-centered-summarization/financial-summarization-pegasus"

In [ ]:
# !rm -r $summarization_model_path/*
# !ls $summarization_model_path

In [ ]:
def load_summarizer(summarizer_name: str, summarization_path: str)->transformers.pipelines.text2text_generation.SummarizationPipeline:
    summarizer_path = os.path.join(summarization_path, summarizer_name)
    if os.path.exists(summarizer_path):
        return pipeline("summarization", summarizer_path, device=torch.cuda.current_device())
    else:
        summarizer = pipeline("summarization", model=summarizer_name, device=torch.cuda.current_device())
        summarizer.save_pretrained(summarizer_path)
        return summarizer


In [ ]:
summarizer = load_summarizer(summarizer_name=summarizer_name, summarization_path=summarization_model_path)

## Summarization Speakers
The functions below are used to leverage the summarization pipeline from the previous section. The database caches results to prevent duplicate runs using the same experimental parameters.

In [ ]:
def split_transcripts_by_speaker(transcripts: List[Transcript]) -> List[List[str]]:
    speaker_divided_transcripts = []
    for transcript in transcripts:
        speaker_divided_transcripts.append(split_transcript_by_speaker(
            transcript
        ))
    return speaker_divided_transcripts

def split_transcript_by_speaker(transcript: Transcript)->List[str]:
    return transcript.transcript.split("\n")



def is_summarized(transcript_summaries: List[TranscriptSummary], transcript: Transcript, summarization_method: str):
    for transcript_summary in transcript_summaries:
        if transcript_summary.summarization_method == summarization_method and transcript.symbol == transcript_summary.symbol and  transcript.date == transcript_summary.date:
            return True
    return False
    

def summarize_text_windows(transcripts: List[Transcript], speaker_window: int, speaker_stride: int, db_util: DB_Util, summarizer: transformers.pipelines.text2text_generation.SummarizationPipeline, summarizer_name: str, logging: bool = False)-> List[TranscriptSummary]:
    summarization_method = f"{summarizer_name}-{speaker_window}-stride-{speaker_stride}"
    transcript_summaries = fetch_all_summaries(db_util=db_util)
    model_max_length = summarizer.tokenizer.model_max_length
    for i, transcript in enumerate(transcripts):
        start_time = time.time()
        print(f"-------------------- Transcript {i+1}/{len(transcripts)}  Company {transcript.symbol}  Transcript Date {transcript.date} --------------------")
        if is_summarized(transcript_summaries=transcript_summaries, transcript=transcript, summarization_method=summarization_method):
            print("Already Summarized")
            continue
        speaker_divided_transcript = split_transcript_by_speaker(transcript)
        if logging:
            print(f"Total Utterances {len(speaker_divided_transcript)}. Total summaries {len(speaker_divided_transcript)/speaker_stride}")
        index = 0
        summarized_windows = []
        try:
            while index < len(speaker_divided_transcript):
                if logging:
                    print(f"Utterance {index + 1} - {index + speaker_window}")
                    print("-----------------------------------------------------------------------")
                summarization_input_string = "\n".join(speaker_divided_transcript[index:min(index+speaker_window, len(speaker_divided_transcript))])
                index += speaker_stride


                # This is probably not the best way to do things. Improve this later
                try:
                    tokens = summarizer.tokenizer(summarization_input_string)
                    input_ids = tokens['input_ids']
                    response = ""
                    # print(len(input_ids))
                    for i in range(0,len(input_ids), model_max_length-24):
                        current_ids = input_ids[i:min(len(input_ids), i + model_max_length-24)]

                        input_string = summarizer.tokenizer.decode(
                            current_ids,
                            skip_special_tokens=True                           
                        )



                        # Removing sentence tokens
                        response += summarizer(
                            input_string,
                        )[0]["summary_text"] + " "

                    # response = summarizer(
                    #     summarization_input_string
                    # )[0]["summary_text"]
                except Exception as e:
                    print("Input too long")
                    print(e)
                    continue

                if logging:    
                    print(summarization_input_string)
                    print(response)
                summarized_windows.append(response.strip())
                


            summary = " ".join(summarized_windows).strip()
            transcript_summary = TranscriptSummary(
                    symbol=transcript.symbol,
                    date=transcript.date,
                    year=transcript.year,
                    quarter=transcript.quarter,
                    summarized_transcript=summary,
                    summarization_method=summarization_method
            )
            status = add_summary(summary=transcript_summary, db_util=db_util)
            if not status:
                print("Failed to insert transcript")
            transcript_summaries.append(
                transcript_summary
            )
        except Exception as e:
            print(e)
            print(f"Unable to summarize transcript Company: {transcript.symbol}  Transcript Date {transcript.date}")
            print(summarization_input_string)
        end_time = time.time()
        print(f"Total time {end_time - start_time}")

    return transcript_summaries

## Running Summarization On Dataset

Running the cells below summarizes the transcripts using the summarization pipeline created above.

In [ ]:
transcripts = fetch_all_transcripts(db_util)

In [ ]:
transcript_summaries = summarize_text_windows(
    transcripts = transcripts,
    speaker_window = 10,
    speaker_stride = 10,
    db_util = db_util,
    summarizer = summarizer,
    summarizer_name = summarizer_name,
    logging = False
)

Token indices sequence length is longer than the specified maximum sequence length for this model (3748 > 512). Running this sequence through the model will result in indexing errors


-------------------- Transcript 1/711  Company AAPL  Transcript Date 2013-01-24 17:00:00 --------------------


/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1070: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
Your max_length is set to 64, but you input_length is only 61. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=30)
Your max_length is set to 64, but you input_length is only 36. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=18)


Total time 14.910606861114502
-------------------- Transcript 2/711  Company AAPL  Transcript Date 2013-04-23 17:00:00 --------------------


Your max_length is set to 64, but you input_length is only 56. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=28)


...

Total time 21.95220685005188
-------------------- Transcript 708/711  Company ABBV  Transcript Date 2022-04-29 11:20:02 --------------------


Your max_length is set to 64, but you input_length is only 61. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=30)
Your max_length is set to 64, but you input_length is only 59. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=29)


Total time 14.791094779968262
-------------------- Transcript 709/711  Company ABBV  Transcript Date 2022-07-29 15:36:11 --------------------
Total time 17.57987666130066
-------------------- Transcript 710/711  Company ABBV  Transcript Date 2022-10-28 12:23:04 --------------------


Your max_length is set to 64, but you input_length is only 25. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=12)
Your max_length is set to 64, but you input_length is only 9. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=4)


Total time 16.696592092514038
-------------------- Transcript 711/711  Company ABBV  Transcript Date 2023-02-09 17:02:03 --------------------
Total time 21.223609924316406


In [ ]:
db_util.con.commit()

In [ ]:
runtime.unassign()